# Kjøre modellen lokalt
### Laste inn modellen fra Hugging Face

In [1]:
# Load model directly
from transformers import AutoModelForSeq2SeqLM, pipeline, T5ForConditionalGeneration


# 1m 22s
model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")


/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
config.json: 100%|██████████| 1.61k/1.61k [00:00<00:00, 6.84MB/s]
pytorch_model.bin: 100%|██████████| 892M/892M [01:18<00:00, 11.4MB/s] 
generation_config.json: 100%|██████████| 147/147 [00:00<00:00, 114kB/s]


In [2]:
from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")



tokenizer_config.json: 100%|██████████| 2.32k/2.32k [00:00<00:00, 23.7MB/s]
spiece.model: 100%|██████████| 792k/792k [00:00<00:00, 13.9MB/s]
tokenizer.json: 100%|██████████| 2.42M/2.42M [00:00<00:00, 9.07MB/s]
special_tokens_map.json: 100%|██████████| 2.20k/2.20k [00:00<00:00, 12.2MB/s]


In [3]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

In [6]:
# input_to_model = "A skier slides down a frictionless slope of height 40m and length 80m. What's the skier's speed at the bottom?"
input_to_model = "The answer to the universe is"
res = pipe(input_to_model, max_new_tokens=(200 + len(input_to_model)), return_tensors=False )
print(res[0]['generated_text'])


The mystery of the universe is solved by: "The answer to the universe is..."


In [7]:

def paraphrase(
    question,
    num_beams=5,
    num_beam_groups=5,
    num_return_sequences=5,
    repetition_penalty=10.0,
    diversity_penalty=3.0,
    no_repeat_ngram_size=2,
    temperature=0.7,
    max_length=128
):
    input_ids = tokenizer(
        f'paraphrase: {question}',
        return_tensors="pt", padding="longest",
        max_length=max_length,
        truncation=True,
    ).input_ids
    
    outputs = model.generate(
        input_ids, temperature=temperature, repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences, no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams, num_beam_groups=num_beam_groups,
        max_length=max_length, diversity_penalty=diversity_penalty
    )

    res = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return res


In [8]:
text = 'What are the best places to see in New York?'
paraphrase(text)


/opt/homebrew/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


['What are some must-see places in New York?',
 'Can you suggest some must-see spots in New York?',
 'Where should one go to experience the best NYC has to offer?',
 'Which places should I visit in New York?',
 'What are the top destinations to explore in New York?']

Her blir den ganske forvirret, og klarer ikke helt å paraphrase korrekt

In [9]:
text = 'translate English to German: How old are you?'
paraphrase(text)

['Can you tell me your age in terms of English?',
 'How old are you?',
 'What is your age in terms of English?',
 'In terms of years, how old are you?',
 'Could you share your age in English?']

Klarer ikke Norsk

In [10]:
text = 'Hei, kan du hjelpe meg med å finne salt'
paraphrase(text)

['Hei, kan du behöver du till finne salt.',
 'HEI: "Can you please let me know how I\'m faring?..."',
 '"Hei, kan du vill till tong til att du jag varfördighed?"',
 'I sprit veya om du att du du gib du till det kommer piget finne?',
 'Hr du fortuit p parin s kommer att du fre?']